In [ ]:
import tensorflow as tf

In [ ]:
!pip install pytorch-transformers

In [ ]:
!pip install pytorch-ignite ipdb

In [ ]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
from pytorch_transformers import cached_path
from pytorch_transformers import AdamW, WarmupLinearSchedule

# GPT2 Related
from  pytorch_transformers import OpenAIGPTTokenizer
from  pytorch_transformers import OpenAIGPTModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
#torch.cuda.get_device_name(0)

In [ ]:
gpttokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [ ]:
gpttokenizer.add_special_tokens({'cls_token': '[CLS]'})

1

In [ ]:
gpttokenizer.convert_tokens_to_ids(gpttokenizer.cls_token)

40478

In [ ]:
df = pd.read_csv("raw_data.csv")

In [ ]:
df.shape

(4072, 6)

In [ ]:
df.sample(10)

,Sno,Date,Headline,Covid,Sentiment,Description
3,3,30-04-20,It's a terrible week for Indian cinema: Rahul ...,0,0,Congress leader Rahul Gandhi condoled Rishi Ka...
3731,3731,07-03-20,DMK General Secretary for 43 yrs Anbazhagan pa...,0,0,Dravida Munnetra Kazhagam (DMK) General Secret...
1434,1434,09-04-20,"Coronavirus cases in Kerala rise to 357, all 8...",1,0,Kerala Chief Minister Pinarayi Vijayan on Wedn...
1014,1014,15-04-20,Man held for social media posts that allegedly...,0,0,The Navi Mumbai Police has arrested a man name...
2902,2902,21-03-20,"5th coronavirus case found in Noida, his socie...",1,0,A person has tested positive for coronavirus i...
1370,1370,10-04-20,Migrant worker donates ₹5k to Kerala CM's reli...,1,1,A migrant worker from Rajasthan contributed ₹5...
3951,3951,03-03-20,24-yr-old with 10k Twitter followers arrested ...,0,0,"A 24-year-old man, who has over 10,000 followe..."
3922,3922,03-03-20,Row in K'taka House over BJP MLA calling 101-y...,0,0,Karnataka Assembly was adjourned on Monday aft...
3239,3239,16-03-20,"No order was placed with presses to print ₹2,0...",0,0,Minister of State for Finance Anurag Singh Tha...
3451,3451,13-03-20,"Fortunate BJP opened doors, received PM Modi's...",0,0,BJP leader Jyotiraditya Scindia said he consid...


In [ ]:
sentences = df.Description.values
labels = df.Sentiment.values


In [ ]:
s0 = sentences[0]

In [ ]:
print(s0)

Rajasthan on Wednesday reported 74 new coronavirus cases, taking the total number of cases in the state to 2,438. All three deaths reported in the state today were from Jaipur, taking the death toll to 55. Notably, 814 patients have recovered in the state and 592 were discharged. Jaipur is the worst-hit district with 878 cases and 30 deaths.


In [ ]:
[gpttokenizer.tokenize(s0)]

[['ra',
  'jas',
  'than</w>',
  'on</w>',
  'wednesday</w>',
  'reported</w>',
  '74</w>',
  'new</w>',
  'coron',
  'a',
  'virus</w>',
  'cases</w>',
  ',</w>',
  'taking</w>',
  'the</w>',
  'total</w>',
  'number</w>',
  'of</w>',
  'cases</w>',
  'in</w>',
  'the</w>',
  'state</w>',
  'to</w>',
  '2</w>',
  ',</w>',
  '4',
  '38</w>',
  '.</w>',
  'all</w>',
  'three</w>',
  'deaths</w>',
  'reported</w>',
  'in</w>',
  'the</w>',
  'state</w>',
  'today</w>',
  'were</w>',
  'from</w>',
  'jai',
  'pu',
  'r</w>',
  ',</w>',
  'taking</w>',
  'the</w>',
  'death</w>',
  'toll</w>',
  'to</w>',
  '55</w>',
  '.</w>',
  'notably</w>',
  ',</w>',
  '8',
  '14</w>',
  'patients</w>',
  'have</w>',
  'recovered</w>',
  'in</w>',
  'the</w>',
  'state</w>',
  'and</w>',
  '5',
  '92</w>',
  'were</w>',
  'discharged</w>',
  '.</w>',
  'jai',
  'pu',
  'r</w>',
  'is</w>',
  'the</w>',
  'worst</w>',
  '-</w>',
  'hit</w>',
  'district</w>',
  'with</w>',
  '8',
  '78</w>',
  'cases</

In [ ]:
tokenized_texts = [gpttokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['ra', 'jas', 'than</w>', 'on</w>', 'wednesday</w>', 'reported</w>', '74</w>', 'new</w>', 'coron', 'a', 'virus</w>', 'cases</w>', ',</w>', 'taking</w>', 'the</w>', 'total</w>', 'number</w>', 'of</w>', 'cases</w>', 'in</w>', 'the</w>', 'state</w>', 'to</w>', '2</w>', ',</w>', '4', '38</w>', '.</w>', 'all</w>', 'three</w>', 'deaths</w>', 'reported</w>', 'in</w>', 'the</w>', 'state</w>', 'today</w>', 'were</w>', 'from</w>', 'jai', 'pu', 'r</w>', ',</w>', 'taking</w>', 'the</w>', 'death</w>', 'toll</w>', 'to</w>', '55</w>', '.</w>', 'notably</w>', ',</w>', '8', '14</w>', 'patients</w>', 'have</w>', 'recovered</w>', 'in</w>', 'the</w>', 'state</w>', 'and</w>', '5', '92</w>', 'were</w>', 'discharged</w>', '.</w>', 'jai', 'pu', 'r</w>', 'is</w>', 'the</w>', 'worst</w>', '-</w>', 'hit</w>', 'district</w>', 'with</w>', '8', '78</w>', 'cases</w>', 'and</w>', '30</w>', 'deaths</w>', '.</w>']


In [ ]:
MAX_LEN = 128

In [ ]:
input_ids = [gpttokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [ ]:
print(input_ids[:2])

[[565, 2623, 815, 504, 9156, 7498, 34604, 783, 19795, 8, 9388, 5767, 240, 1493, 481, 5063, 2253, 498, 5767, 500, 481, 2720, 485, 280, 240, 44, 15931, 239, 589, 1203, 9020, 7498, 500, 481, 2720, 1873, 641, 617, 8199, 673, 255, 240, 1493, 481, 1767, 14164, 485, 20218, 239, 36792, 240, 54, 7765, 9553, 604, 7975, 500, 481, 2720, 488, 46, 32964, 641, 22014, 239, 8199, 673, 255, 544, 481, 3720, 260, 1751, 9141, 556, 54, 24210, 5767, 488, 6828, 9020, 239], [481, 5063, 2253, 498, 19795, 8, 9388, 5767, 500, 1761, 3569, 980, 9331, 485, 281, 240, 44, 19755, 861, 39, 10353, 783, 5767, 641, 7498, 1873, 239, 2578, 240, 481, 1767, 14164, 1978, 485, 26836, 861, 867, 783, 9020, 641, 7498, 1873, 239, 481, 2253, 498, 4613, 5767, 500, 1761, 3569, 4276, 491, 280, 240, 42, 38794, 240, 1000, 277, 240, 48, 32964, 588, 2544, 260, 11252, 9553, 604, 7975, 500, 481, 5670, 9416, 620, 1311, 239]]


In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN-1, dtype="long", truncating="post", padding="post")

In [ ]:
input_ids = [np.append(x, gpttokenizer.convert_tokens_to_ids('[CLS]'))  for x in input_ids]

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.05)

In [ ]:
# Convert all data into torch tensors bec it is the the required type for the pytorch model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

In [ ]:
print(train_inputs.shape)

torch.Size([3868, 128])


In [ ]:
print(train_inputs[0])

tensor([  481,  5063,  2253,   498, 19795,     8,  9388,  5767,   500, 11203,
          256,   252,  3720,   260,  1751,  2720, 29901,  3867,  3341,  1455,
           44,  9509,   557, 34534,   725,   989,  9587,  6353,   239,   481,
         2561, 17156, 18905,   989,   763,   604,   694, 22014,   488, 11252,
         9020,   239,   498,   481,   783,  5767,  7498,  1873,   240, 23339,
          641, 11233,   500, 26218, 27563,   239,  8676,   240,   588,  2544,
          260, 11252,  5767,   500,   632, 13047,   813, 13723,   488,  5000,
        11272,  1978,   485, 18688,   291,   488, 27277,   287, 36006,  1873,
          239,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0, 40478])

In [ ]:
batch_size = 16

# Training
train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Validation
validation_data = TensorDataset(validation_inputs, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
class GPTWithClfHead(nn.Module):
    def __init__(self, config, adapt_args, gptmodel):
        super().__init__()
        self.transformer = gptmodel
        self.num_labels = adapt_args.num_classes
        self.classifier = nn.Linear(config['n_embd'], adapt_args.num_classes)

    
    def forward(self, input_ids, mc_token_ids=None, labels=None, token_type_ids=None,
                position_ids=None, past=None, head_mask=None):
        transformer_outputs = self.transformer(input_ids, position_ids=position_ids, token_type_ids=token_type_ids
                                               , head_mask=head_mask)
        hidden_states = transformer_outputs[0]
        

        # last token of the sequence is classification token
        token_ids = torch.full_like(hidden_states[..., :1, :], hidden_states.shape[-2]-1, dtype=torch.long)
        output = hidden_states.gather(-2, token_ids).squeeze(-2) # shape (bsz, n_embd)

        clf_logits = self.classifier(output)
        outputs = (clf_logits,) + transformer_outputs[1:]
        if labels is not None:
          loss_fct = CrossEntropyLoss()
          loss = loss_fct(clf_logits.view(-1, self.num_labels), labels.view(-1))
          outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
from collections import namedtuple
AdaptationConfig = namedtuple('AdaptationConfig',
  field_names="num_classes, dropout, initializer_range, batch_size, lr, max_norm, n_epochs,"
              "n_warmup, valid_set_prop, gradient_accumulation_steps, device,"
              "log_dir")
adapt_args = AdaptationConfig(
               2          , 0.1    , 0.02             , 16        , 6.5e-5, 1.0   , 4,
               10      , 0.1           , 1, device,
               "./" )

In [ ]:
gptconfig = cached_path("https://s3.amazonaws.com/models.huggingface.co/bert/openai-gpt-config.json")

In [ ]:
import json
with open(gptconfig, "r") as read_file:
    gptconfig = json.load(read_file)
print (gptconfig)

{'afn': 'gelu', 'architectures': ['OpenAIGPTLMHeadModel'], 'attn_pdrop': 0.1, 'embd_pdrop': 0.1, 'initializer_range': 0.02, 'layer_norm_epsilon': 1e-05, 'model_type': 'openai-gpt', 'n_ctx': 512, 'n_embd': 768, 'n_head': 12, 'n_layer': 12, 'n_positions': 512, 'n_special': 0, 'predict_special_tokens': True, 'resid_pdrop': 0.1, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'summary_type': 'cls_index', 'summary_use_proj': True, 'task_specific_params': {'text-generation': {'do_sample': True, 'max_length': 50}}, 'vocab_size': 40478}


In [ ]:
gptmodel = OpenAIGPTModel.from_pretrained('openai-gpt')

In [ ]:
gptmodel.resize_token_embeddings(len(gpttokenizer))

Embedding(40479, 768)

In [ ]:
model = GPTWithClfHead(gptconfig, adapt_args, gptmodel)
#model.cuda()

In [ ]:
import os
from torch.utils.data import DataLoader
from ignite.engine import Engine, Events
from ignite.metrics import RunningAverage, Accuracy
from ignite.handlers import ModelCheckpoint
from ignite.contrib.handlers import CosineAnnealingScheduler, PiecewiseLinear, create_lr_scheduler_with_warmup, ProgressBar
optimizer = torch.optim.Adam(model.parameters(), lr=adapt_args.lr)

train_loss_set = []

# Training function and trainer
def update(engine, batch):
    model.train()
    b_input_ids, labels = (t.to(adapt_args.device) for t in batch)
    outputs = model(input_ids = b_input_ids, labels=labels)
    loss = outputs[0]
    
    loss = loss / adapt_args.gradient_accumulation_steps
    train_loss_set.append(loss.item())
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), adapt_args.max_norm)
    if engine.state.iteration % adapt_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
    return loss.item()
trainer = Engine(update)

# Evaluation function and evaluator (evaluator output is the input of the metrics)
def inference(engine, batch):
    model.eval()
    with torch.no_grad():
      b_input_ids, labels = (t.to(adapt_args.device) for t in batch)
      outputs = model(input_ids = b_input_ids)
      logits = outputs[0]
    return logits, labels
evaluator = Engine(inference)

# Attach metric to evaluator & evaluation to trainer: evaluate on valid set after each epoch
Accuracy().attach(evaluator, "accuracy")
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(validation_dataloader)
    print(f"Validation Epoch: {engine.state.epoch} accuracy: {100*(evaluator.state.metrics['accuracy'])}")
     

# Learning rate schedule: linearly warm-up to lr and then to zero
scheduler = PiecewiseLinear(optimizer, 'lr', [(0, 0.0), (adapt_args.n_warmup, adapt_args.lr),
                                              (len(train_dataloader)*adapt_args.n_epochs, 0.0)])
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)

# Add progressbar with loss
RunningAverage(output_transform=lambda x: x).attach(trainer, "loss")
ProgressBar(persist=True).attach(trainer, metric_names=['loss'])

# @trainer.on(Events.ITERATION_COMPLETED)
@trainer.on(Events.EPOCH_COMPLETED)
def log_running_avg_metrics(engine):    
    print(f" Epoch: {engine.state.epoch} training loss: {engine.state.metrics['loss']}")
    

# Save checkpoints and finetuning config
checkpoint_handler = ModelCheckpoint(adapt_args.log_dir, 'finetuning_checkpoint', save_interval=1, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'mymodel': model})

/usr/local/lib/python3.6/dist-packages/ignite/handlers/checkpoint.py:708: UserWarning: Argument save_interval is deprecated and should be None. This argument will be removed in 0.5.0.Please, use events filtering instead, e.g. Events.ITERATION_STARTED(every=1000)
  warnings.warn(msg)


In [ ]:
trainer.run(train_dataloader, max_epochs=adapt_args.n_epochs)

Validation Epoch: 1 accuracy: 87.25490196078431

 Epoch: 1 training loss: 0.41782893266026316


Validation Epoch: 2 accuracy: 83.82352941176471

 Epoch: 2 training loss: 0.24038942854549308


Validation Epoch: 3 accuracy: 85.7843137254902

 Epoch: 3 training loss: 0.11850326717922115


Validation Epoch: 4 accuracy: 85.29411764705883

 Epoch: 4 training loss: 0.039134350135139266


State:
	iteration: 968
	epoch: 4
	epoch_length: 242
	max_epochs: 4
	output: 0.008870438672602177
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: torch.utils.data.dataloader.DataLoader
	seed: <class 'NoneType'>
	times: <class 'dict'>